In [9]:
from imutils import contours
from skimage import measure
import numpy as np
import imutils
import cv2
import pandas as pd
import glob
import swifter

import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

In [10]:
df = pd.read_csv('exceptions.csv')

In [11]:
def get_folder(path):
    return path.split('/')[-2]

def get_file(path):
    return path.split('/')[-1]

In [12]:
df['folder'] = df.path.swifter.progress_bar(True).apply(get_folder)

In [13]:
df['file'] = df.path.swifter.progress_bar(True).apply(get_file)

In [14]:
df

,path,raises_exception,folder,file
0,dataset/ocular/0/2342_left.jpg,True,0,2342_left.jpg
1,dataset/ocular/0/3407_right.jpg,True,0,3407_right.jpg
2,dataset/ocular/0/2343_left.jpg,False,0,2343_left.jpg
3,dataset/ocular/0/3294_right.jpg,True,0,3294_right.jpg
4,dataset/ocular/0/940_right.jpg,False,0,940_right.jpg
...,...,...,...,...
3419,dataset/ocular/1/1964_left.jpg,True,1,1964_left.jpg
3420,dataset/ocular/1/1265_right.jpg,True,1,1265_right.jpg
3421,dataset/ocular/1/1965_left.jpg,False,1,1965_left.jpg
3422,dataset/ocular/1/1503_right.jpg,True,1,1503_right.jpg


In [21]:
def get_disk(row):
  if (row.raises_exception=="False"):
    return None
    
  try:
    image = row.path
    folder = row.folder
    file = row.file
    image = cv2.imread(image)
    
    width = 512
    dim = (width, width)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    crop = 100
    image = image[100:width-100, 100:width-100]
    
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.bilateralFilter(gray,9,75,75)
    median=cv2.medianBlur(blur, 5)

    # threshold the image to reveal light regions in the
    # blurred image
    thresh = cv2.threshold(median, 155, 255, cv2.THRESH_BINARY)[1]
    # perform a series of erosions and dilations to remove
    # any small blobs of noise from the thresholded image
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=4)

    # perform a connected component analysis on the thresholded
    # image, then initialize a mask to store only the "large"
    # components
    labels = measure.label(thresh, connectivity=2, background=0)
    mask = np.zeros(thresh.shape, dtype="uint8")

    # loop over the unique components
    for label in np.unique(labels):
      # if this is the background label, ignore it
      if label == 0:
        continue

      # otherwise, construct the label mask and count the
      # number of pixels 
      labelMask = np.zeros(thresh.shape, dtype="uint8")
      labelMask[labels == label] = 255
      numPixels = cv2.countNonZero(labelMask)

      # if the number of pixels in the component is sufficiently
      # large, then add it to our mask of "large blobs"
      if numPixels >300:
        mask = cv2.add(mask, labelMask)

    # find the contours in the mask, then sort them from left to
    # right
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
      cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = contours.sort_contours(cnts)[0]
    # print (cnts)
    # loop over the contours
    for (i, c) in enumerate(cnts):
        ellipse = cv2.fitEllipse(c)
        (x, y, w, h) = cv2.boundingRect(c)
        break
      # draw the bright spot on the image
    padding = 0
    image_0 = image[y-padding:y+h+padding, x-padding:x+w+padding]
    color_coverted = cv2.cvtColor(image_0, cv2.COLOR_BGR2RGB)
    pil_image=Image.fromarray(color_coverted)
    pil_image.save("disk_dataset_ocular_0/"+str(folder)+"/"+file)
    
    padding = 100
    image_100 = image[y-padding:y+h+padding, x-padding:x+w+padding]
    color_coverted = cv2.cvtColor(image_100, cv2.COLOR_BGR2RGB)
    pil_image=Image.fromarray(color_coverted)
    pil_image.save("disk_dataset_ocular_100/"+str(folder)+"/"+file)
  except:
    print (row.file)
    pass

In [22]:
df.head()

,path,raises_exception,folder,file
0,dataset/ocular/0/2342_left.jpg,True,0,2342_left.jpg
1,dataset/ocular/0/3407_right.jpg,True,0,3407_right.jpg
2,dataset/ocular/0/2343_left.jpg,False,0,2343_left.jpg
3,dataset/ocular/0/3294_right.jpg,True,0,3294_right.jpg
4,dataset/ocular/0/940_right.jpg,False,0,940_right.jpg


In [23]:
df.swifter.progress_bar(True).apply(get_disk, axis=1)

(pid=14899) 951_left.jpg
(pid=14899) 2342_left.jpg
(pid=14899) 3407_right.jpg
(pid=14899) 2343_left.jpg
(pid=14899) 940_right.jpg
(pid=14899) 859_left.jpg
(pid=14899) 3276_left.jpg
(pid=14902) 2662_right.jpg
(pid=14899) 1117_left.jpg
(pid=14904) 2417_left.jpg
(pid=14905) 259_left.jpg
(pid=14900) 2341_left.jpg
(pid=14906) 4618_left.jpg
(pid=14902) 2788_left.jpg
(pid=14904) 2887_left.jpg
(pid=14899) 3010_left.jpg
(pid=14900) 2644_right.jpg
(pid=14906) 2777_left.jpg
(pid=14900) 3189_left.jpg
(pid=14899) 3_left.jpg
(pid=14903) 2924_left.jpg
(pid=14901) 3091_left.jpg
(pid=14902) 2512_right.jpg
(pid=14905) 4355_left.jpg
(pid=14903) 2925_left.jpg
(pid=14904) 2671_left.jpg
(pid=14900) 2340_left.jpg
(pid=14906) 2981_left.jpg
(pid=14902) 2789_left.jpg
(pid=14906) 432_left.jpg
(pid=14903) 496_left.jpg
(pid=14902) 1620_left.jpg
(pid=14901) 4336_left.jpg
(pid=14904) 4578_left.jpg
(pid=14899) 2771_right.jpg
(pid=14905) 1393_left.jpg
(pid=14900) 2969_right.jpg
(pid=14906) 2511_left.jpg
(pid=14904) 26

(pid=14905) 2666_left.jpg
(pid=14903) 2600_left.jpg
(pid=14899) 2439_left.jpg
(pid=14905) 2667_left.jpg
(pid=14900) 670_right.jpg
(pid=14901) 658_left.jpg
(pid=14902) 2848_left.jpg
(pid=14904) 2940_right.jpg
(pid=14902) 2527_right.jpg
(pid=14903) 2466_left.jpg
(pid=14901) 3332_right.jpg
(pid=14900) 3306_right.jpg
(pid=14906) 4271_left.jpg
(pid=14903) 3261_left.jpg
(pid=14905) 2891_left.jpg
(pid=14902) 2849_left.jpg
(pid=14900) 3130_right.jpg
(pid=14899) 2438_left.jpg
(pid=14905) 522_left.jpg
(pid=14906) 2866_right.jpg
(pid=14902) 2461_right.jpg
(pid=14906) 2927_left.jpg
(pid=14905) 2401_left.jpg
(pid=14903) 3007_left.jpg
(pid=14901) 3428_left.jpg
(pid=14901) 2933_right.jpg
(pid=14905) 2720_right.jpg
(pid=14900) 3406_left.jpg
(pid=14904) 3258_left.jpg
(pid=14906) 804_left.jpg
(pid=14900) 2838_left.jpg
(pid=14902) 2657_right.jpg
(pid=14901) 2816_left.jpg
(pid=14906) 2920_right.jpg
(pid=14900) 3407_left.jpg
(pid=14899) 2659_right.jpg
(pid=14903) 3006_left.jpg
(pid=14905) 2160_left.jpg
(pi

(pid=14899) 3405_left.jpg
(pid=14902) 4464_left.jpg
(pid=14903) 2676_left.jpg
(pid=14902) 3065_right.jpg
(pid=14904) 2557_left.jpg
(pid=14903) 2407_right.jpg
(pid=14901) 3135_left.jpg
(pid=14903) 2880_left.jpg
(pid=14899) 589_left.jpg
(pid=14904) 832_right.jpg
(pid=14901) 1032_left.jpg
(pid=14901) 4292_left.jpg
(pid=14905) 3089_right.jpg
(pid=14900) 2545_left.jpg
(pid=14906) 2603_left.jpg
(pid=14903) 2410_left.jpg
(pid=14904) 346_left.jpg
(pid=14900) 2723_left.jpg
(pid=14899) 165_right.jpg
(pid=14901) 3353_left.jpg
(pid=14905) 3330_right.jpg
(pid=14901) 1033_left.jpg
(pid=14906) 2465_left.jpg
(pid=14903) 2411_left.jpg
(pid=14902) 3253_right.jpg
(pid=14901) 3352_left.jpg
(pid=14899) 1503_left.jpg
(pid=14902) 987_right.jpg
(pid=14905) 2395_left.jpg
(pid=14906) 189_left.jpg
(pid=14901) 3134_left.jpg
(pid=14902) 4465_left.jpg
(pid=14905) 1429_right.jpg
(pid=14906) 3209_right.jpg
(pid=14901) 2733_left.jpg
(pid=14899) 3249_right.jpg
(pid=14902) 429_left.jpg
(pid=14902) 3297_left.jpg
(pid=149

(pid=14905) 2575_left.jpg
(pid=14902) 213_left.jpg
(pid=14904) 187_left.jpg
(pid=14899) 2602_right.jpg
(pid=14902) 2705_right.jpg
(pid=14904) 3303_right.jpg
(pid=14906) 2811_left.jpg
(pid=14901) 1255_right.jpg
(pid=14905) 457_left.jpg
(pid=14903) 2097_left.jpg
(pid=14899) 2547_left.jpg
(pid=14902) 2909_left.jpg
(pid=14906) 2915_right.jpg
(pid=14905) 2574_left.jpg
(pid=14899) 1021_left.jpg
(pid=14903) 2375_right.jpg
(pid=14905) 2712_left.jpg
(pid=14902) 2599_left.jpg
(pid=14903) 3038_left.jpg
(pid=14904) 3135_right.jpg
(pid=14899) 3340_left.jpg
(pid=14903) 2972_right.jpg
(pid=14902) 2533_right.jpg
(pid=14901) 1272_left.jpg
(pid=14899) 3126_left.jpg
(pid=14900) 2337_left.jpg
(pid=14901) 1307_right.jpg
(pid=14899) 2869_right.jpg
(pid=14905) 843_right.jpg
(pid=14906) 933_left.jpg
(pid=14902) 2598_left.jpg
(pid=14900) 2723_right.jpg
(pid=14899) 3127_left.jpg
(pid=14903) 2459_left.jpg
(pid=14904) 2419_right.jpg
(pid=14906) 2810_left.jpg
(pid=14900) 3333_right.jpg
(pid=14904) 2960_left.jpg
(p

(pid=14901) 1325_left.jpg
(pid=14900) 3057_left.jpg
(pid=14899) 25_left.jpg
(pid=14906) 343_left.jpg
(pid=14905) 3163_left.jpg
(pid=14901) 95_left.jpg
(pid=14905) 3305_left.jpg
(pid=14900) 3056_left.jpg
(pid=14903) 2703_left.jpg
(pid=14899) 4426_right.jpg
(pid=14902) 3193_left.jpg
(pid=14905) 2992_left.jpg
(pid=14903) 3443_right.jpg
(pid=14904) 2928_left.jpg
(pid=14899) 3293_left.jpg
(pid=14902) 3192_left.jpg
(pid=14903) 620_left.jpg
(pid=14905) 2502_left.jpg
(pid=14903) 2702_left.jpg
(pid=14901) 1334_right.jpg
(pid=14899) 4352_left.jpg
(pid=14906) 2656_right.jpg
(pid=14900) 4197_left.jpg
(pid=14906) 124_left.jpg
(pid=14899) 3030_right.jpg
(pid=14902) 2469_left.jpg
(pid=14906) 2858_left.jpg
(pid=14901) 1352_left.jpg
(pid=14900) 3230_left.jpg
(pid=14904) 3225_left.jpg
(pid=14905) 2764_left.jpg
(pid=14902) 2837_right.jpg
(pid=14901) 1353_left.jpg
(pid=14900) 1151_left.jpg
(pid=14906) 2526_right.jpg
(pid=14899) 24_left.jpg
(pid=14902) 2468_left.jpg
(pid=14900) 3080_right.jpg
(pid=14905) 2

...

(pid=14900) 2710_left.jpg
(pid=14901) 1301_left.jpg
(pid=14902) 3250_left.jpg
(pid=14903) 2453_left.jpg
(pid=14902) 2563_right.jpg
(pid=14901) 1406_left.jpg
(pid=14903) 570_left.jpg
(pid=14904) 3420_left.jpg
(pid=14905) 376_left.jpg
(pid=14900) 633_left.jpg
(pid=14906) 4237_right.jpg
(pid=14903) 3195_right.jpg
(pid=14900) 2576_left.jpg
(pid=14903) 716_left.jpg
(pid=14906) 107_left.jpg
(pid=14901) 1453_right.jpg
(pid=14900) 2437_right.jpg
(pid=14906) 3445_left.jpg
(pid=14901) 1209_right.jpg
(pid=14899) 2462_right.jpg
(pid=14905) 236_right.jpg
(pid=14902) 3036_left.jpg
(pid=14900) 2601_right.jpg
(pid=14900) 2577_left.jpg
(pid=14906) 3284_right.jpg
(pid=14905) 1435_right.jpg
(pid=14900) 2711_left.jpg
(pid=14902) 2631_left.jpg
(pid=14903) 4256_right.jpg
(pid=14901) 1334_left.jpg
(pid=14904) 2397_right.jpg
(pid=14906) 3444_left.jpg
(pid=14903) 2634_left.jpg
(pid=14906) 3377_left.jpg
(pid=14904) 4348_left.jpg
(pid=14900) 3949_left.jpg
(pid=14903) 2452_left.jpg
(pid=14899) 3089_left.jpg
(pid=

(pid=14902) 434_left.jpg
(pid=14905) 3338_right.jpg
(pid=14901) 1318_left.jpg
(pid=14904) 2874_left.jpg
(pid=14903) 2329_left.jpg
(pid=14901) 1287_right.jpg
(pid=14899) 3303_left.jpg
(pid=14900) 3306_left.jpg
(pid=14903) 3387_left.jpg
(pid=14905) 3983_left.jpg
(pid=14906) 3019_left.jpg
(pid=14901) 167_right.jpg
(pid=14903) 2774_left.jpg
(pid=14901) 167_left.jpg
(pid=14902) 3310_left.jpg
(pid=14900) 4608_left.jpg
(pid=14906) 3018_left.jpg
(pid=14905) 364_right.jpg
(pid=14899) 3302_left.jpg
(pid=14906) 2970_right.jpg
(pid=14903) 2512_left.jpg
(pid=14899) 3164_left.jpg
(pid=14904) 2682_left.jpg
(pid=14906) 2479_left.jpg
(pid=14901) 1210_left.jpg
(pid=14900) 2991_left.jpg
(pid=14902) 3176_left.jpg
(pid=14903) 2983_left.jpg
(pid=14900) 2501_left.jpg
(pid=14901) 1211_left.jpg
(pid=14899) 2763_left.jpg
(pid=14900) 2767_left.jpg
(pid=14904) 2388_right.jpg
(pid=14905) 2387_left.jpg
(pid=14899) 2505_left.jpg
(pid=14902) 3177_left.jpg
(pid=14906) 2478_left.jpg
(pid=14901) 153_left.jpg
(pid=14903)

(pid=14902) 1134_right.jpg
(pid=14906) 3393_left.jpg
(pid=14904) 2537_left.jpg
(pid=14905) 2403_right.jpg
(pid=14901) 1281_right.jpg
(pid=14905) 2918_right.jpg
(pid=14899) 2597_left.jpg
(pid=14904) 3330_left.jpg
(pid=14900) 2687_right.jpg
(pid=14903) 4248_right.jpg
(pid=14901) 1270_left.jpg
(pid=14905) 2637_left.jpg
(pid=14899) 2906_left.jpg
(pid=14902) 3131_right.jpg
(pid=14901) 1271_left.jpg
(pid=14905) 2451_left.jpg
(pid=14900) 3129_left.jpg
(pid=14906) 884_right.jpg
(pid=14905) 241_left.jpg
(pid=14900) 2842_left.jpg
(pid=14905) 3319_right.jpg
(pid=14906) 2904_left.jpg
(pid=14905) 
(pid=14901) 1374_right.jpg
(pid=14905) 2362_left.jpg
(pid=14904) 3156_left.jpg
(pid=14903) 2389_left.jpg
(pid=14901) 1378_left.jpg
(pid=14906) 2594_left.jpg
(pid=14905) 2363_left.jpg
(pid=14899) 2846_left.jpg
(pid=14902) 3077_right.jpg
(pid=14900) 422_right.jpg
(pid=14899) 2331_right.jpg
(pid=14906) 2905_left.jpg
(pid=14903) 3327_left.jpg
(pid=14899) 2847_left.jpg
(pid=14902) 4182_right.jpg
(pid=14900) 10

(pid=14906) 2620_left.jpg
(pid=14902) 3019_right.jpg
(pid=14904) 3121_left.jpg
(pid=14900) 2474_left.jpg
(pid=14901) 1415_right.jpg
(pid=14906) 2446_left.jpg
(pid=14903) 2460_left.jpg
(pid=14900) 548_right.jpg
(pid=14905) 4023_left.jpg
(pid=14906) 2424_right.jpg
(pid=14906) 
(pid=14899) 1041_left.jpg
(pid=14904) 3347_left.jpg
(pid=14903) 2461_left.jpg
(pid=14900) 2475_left.jpg
(pid=14903) 2636_right.jpg
(pid=14901) 1881_right.jpg
(pid=14902) 3264_left.jpg
(pid=14905) 4576_left.jpg
(pid=14900) 2613_left.jpg
(pid=14902) 3002_left.jpg
(pid=14905) 2419_left.jpg
(pid=14906) 1120_left.jpg
(pid=14904) 2554_right.jpg
(pid=14900) 3178_right.jpg
(pid=14901) 1286_right.jpg
(pid=14905) 2889_left.jpg
(pid=14906) 3241_left.jpg
(pid=14904) 3346_left.jpg
(pid=14902) 2969_left.jpg
(pid=14905) 4577_left.jpg
(pid=14901) 1978_left.jpg
(pid=14899) 3147_left.jpg
(pid=14906) 3027_left.jpg
(pid=14902) 2703_right.jpg
(pid=14904) 4287_left.jpg
(pid=14901) 1473_left.jpg
(pid=14903) 2607_left.jpg
(pid=14899) 2740

0       None
1       None
2       None
3       None
4       None
        ... 
3419    None
3420    None
3421    None
3422    None
3423    None
Length: 3424, dtype: object